# nnUNet, training, 2D, 5-fold

In [ ]:
import os
import json
import shutil

In [ ]:
!pip install nnunetv2

In [ ]:
!pip install -U triton

In [ ]:
# Define dataset details
task_name = 'Dataset001_spine'
base_dir = 'PATH/nnUNet_raw_data'  # Replace 'PATH' with the actual base directory
task_dir = os.path.join(base_dir, task_name)
imagesTr_dir = os.path.join(task_dir, 'imagesTr')
labelsTr_dir = os.path.join(task_dir, 'labelsTr')

# Create necessary directories if they don’t exist
os.makedirs(imagesTr_dir, exist_ok=True)
os.makedirs(labelsTr_dir, exist_ok=True)

In [ ]:
# Define dataset configuration in a JSON format
dataset_json = {
    "channel_names": {
        "0": "CT"  # The dataset has only one channel (CT scan images)
    },
    "labels": {
        "background": 0,  # Label for the background
        "class1": 1  # Label for the first class (modify if more classes exist)
    },
    "numTraining": len(os.listdir(imagesTr_dir)),  # Count the number of training images
    "file_ending": ".nii.gz"  # Medical image file format (NIfTI)
}

# Save the JSON configuration file inside the dataset directory
with open(os.path.join(task_dir, 'dataset.json'), 'w') as f:
    json.dump(dataset_json, f, indent=4)  # Write JSON with indentation for readability

# Print confirmation message
print(f"dataset.json created in {task_dir}")

In [ ]:
# Define environment variables for nnUNet's required paths
os.environ['nnUNet_raw'] = 'PATH/nnUNet_raw_data'
os.environ['nnUNet_preprocessed'] = 'PATH/nnUNet_preprocessed'
os.environ['nnUNet_results'] = 'PATH/nnUNet_results'

# Ensure directories exist by creating them if they don't
os.makedirs(os.environ['nnUNet_raw'], exist_ok=True)
os.makedirs(os.environ['nnUNet_preprocessed'], exist_ok=True)
os.makedirs(os.environ['nnUNet_results'], exist_ok=True)

In [ ]:
# Display available options for nnUNet's preprocessing command
!nnUNetv2_plan_and_preprocess -h

In [ ]:
# Convert the dataset for nnUNet training
!nnUNetv2_plan_and_preprocess -d 001 -pl nnUNetPlannerResEncL --verify_dataset_integrity

In [ ]:
# Display help menu for the nnUNet training command
!nnUNetv2_train -h

In [ ]:
# Start training the nnUNet model
# Arguments:
# - Dataset name: Dataset001_spine
# - Configuration: 2D (2D network training)
# - Fold: 0 (First fold of cross-validation)
!nnUNetv2_train Dataset001_spine 2d 0

In [ ]:
#Fold: 1
!nnUNetv2_train 001 2d 1 -p nnUNetResEncUNetLPlans --npz -tr nnUNetTrainer_500epochs

In [ ]:
#Fold: 2
!nnUNetv2_train 001 2d 2 -p nnUNetResEncUNetLPlans --npz -tr nnUNetTrainer_500epochs

In [ ]:
#Fold: 3
!nnUNetv2_train 001 2d 3 -p nnUNetResEncUNetLPlans --npz -tr nnUNetTrainer_500epochs

In [ ]:
#Fold: 4
!nnUNetv2_train 001 2d 4 -p nnUNetResEncUNetLPlans --npz -tr nnUNetTrainer_500epochs